In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python --version

Python 3.10.12


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model2.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
year = 2023
month = 3
taxi_type = 'yellow'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'./output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
y_pred.std()

np.float64(6.247488852238703)

In [9]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

In [10]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)